# Backtest of Technical Analysis Based Strategies | DMAC
---

__Authors:__ Emily Bertani, Max Acheson, Josh Mischung  
__Data Source:__ Yahoo Finance accessed by `pandas-datareader` and `yfinance`  
__Strategy:__ DMAC 
__Instrument(s):__ SPY  

__Intended Use:__  
The primary objective of this notebook is to determine the profitability and optimum window sizes of the DMAC strategy.

<br>

## Imports & Functions
---
__Imports__

In [5]:
# Supress bokeh warnings generated by backtesting
import warnings
warnings.filterwarnings('ignore')

import backtesting

In [18]:
import pandas as pd
import pandas_datareader.data as pdr
import yfinance as yf

from backtesting import Backtest, Strategy
from backtesting.lib import crossover

<br>

__Functions & Classes__

In [15]:
def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()


class SmaCross(Strategy):
    n1=10
    n2=20
    
    def init(self):
        price = self.data.Close
        self.ma1 = self.I(SMA, self.data.Close, self.n1)
        self.ma2 = self.I(SMA, self.data.Close, self.n2)
        
    def next(self):
        if crossover(self.ma1, self.ma2):
            self.position.close()
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.position.close()
            self.sell()

<br>

## Retreive and Format Data
---

In [13]:
# Set variables
ticker = 'AAPL'
start_date = '2015-05-01'
end_date = '2022-03-25'

# Create OHLCV DataFrame
stock_df = pdr.get_data_yahoo(
    ticker,
    start_date,
    end_date
)

stock_df.drop(columns=['Close'], inplace=True)
stock_df.rename(columns={'Adj Close' : 'Close'}, inplace=True)
stock_df.insert(3, 'Close', stock_df.pop('Close'))

stock_df.head()

,High,Low,Open,Close,Volume
Date,,,,,
2015-05-01,32.532501,31.325001,31.525000,29.264734,234050400.0
2015-05-04,32.642502,32.064999,32.375000,29.207998,203953200.0
2015-05-05,32.112499,31.445000,32.037498,28.549854,197085600.0
2015-05-06,31.687500,30.840000,31.639999,28.370567,288564000.0
2015-05-07,31.520000,31.004999,31.192499,28.546047,175763600.0


<br>

## Evaluate and Plot Financial Metrics
---

In [16]:
bt = Backtest(stock_df, SmaCross, cash=10_000, commission=.002)
stats = bt.run()
stats

Start                     2015-05-01 00:00:00
End                       2022-03-25 00:00:00
Duration                   2520 days 00:00:00
Exposure Time [%]                   98.389879
Equity Final [$]                 28677.129808
Equity Peak [$]                  42434.075594
Return [%]                         186.771298
Buy & Hold Return [%]              497.032591
Return (Ann.) [%]                   16.493553
Volatility (Ann.) [%]               47.715907
Sharpe Ratio                         0.345662
Sortino Ratio                         0.62189
Calmar Ratio                         0.375381
Max. Drawdown [%]                  -43.938146
Avg. Drawdown [%]                   -6.920735
Max. Drawdown Duration      497 days 00:00:00
Avg. Drawdown Duration       48 days 00:00:00
# Trades                                   67
Win Rate [%]                        40.298507
Best Trade [%]                      79.788108
Worst Trade [%]                    -14.188268
Avg. Trade [%]                    

In [17]:
bt.plot();

<br>

## Optimize Window Sizes
---

In [19]:
stats = bt.optimize(n1=range(5, 30, 5),
                    n2=range(10, 70, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
stats

Backtest.optimize:   0%|          | 0/17 [00:00<?, ?it/s]

Start                     2015-05-01 00:00:00
End                       2022-03-25 00:00:00
Duration                   2520 days 00:00:00
Exposure Time [%]                   98.619896
Equity Final [$]                 70013.440179
Equity Peak [$]                  72425.075812
Return [%]                         600.134402
Buy & Hold Return [%]              497.032591
Return (Ann.) [%]                   32.579377
Volatility (Ann.) [%]               62.423436
Sharpe Ratio                         0.521909
Sortino Ratio                          1.0963
Calmar Ratio                         0.961437
Max. Drawdown [%]                  -33.886117
Avg. Drawdown [%]                   -6.009762
Max. Drawdown Duration      450 days 00:00:00
Avg. Drawdown Duration       31 days 00:00:00
# Trades                                  102
Win Rate [%]                        47.058824
Best Trade [%]                      51.761384
Worst Trade [%]                    -15.693983
Avg. Trade [%]                    

<br>

__Optimal Window Parameters__

In [29]:
# Recover optimal windows
n1 = stats._strategy.n1
n2 = stats._strategy.n2
print(
    f"The optimal window size for the short window is {n1}.\n" +
    f"The optimal window size for the long window is {n2}."
)

The optimal window size for the short window is 5.
The optimal window size for the long window is 15.


In [31]:
bt.plot(plot_volume=False, plot_pl=False);

<br>

__Trade Data__

In [32]:
stats['_trades'] 

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-309,24,53,32.310250,32.270000,12.437109,0.001246,2015-06-05,2015-07-17,42 days
1,309,53,60,32.334540,30.844999,-460.268214,-0.046067,2015-07-17,2015-07-28,11 days
2,-310,60,87,30.783309,28.122499,824.851053,0.086437,2015-07-28,2015-09-03,37 days
3,368,87,89,28.178744,27.937500,-88.777963,-0.008561,2015-09-03,2015-09-08,5 days
4,-368,89,91,27.881625,27.567499,115.598309,0.011266,2015-09-08,2015-09-10,2 days
...,...,...,...,...,...,...,...,...,...,...
97,390,1651,1687,154.017427,172.320007,7138.006433,0.118834,2021-11-18,2022-01-11,54 days
98,-390,1687,1703,171.975367,174.479996,-976.805083,-0.014564,2022-01-11,2022-02-03,23 days
99,378,1703,1713,174.828956,171.029999,-1436.005723,-0.021730,2022-02-03,2022-02-17,14 days
100,-379,1713,1735,170.687939,165.509995,1962.440880,0.030336,2022-02-17,2022-03-22,33 days
